From: https://github.com/rahulbot/notebook-examples

1. Modify C2 + C3, change path to another year
2. Modify C5, change state name

Loading and Analyzing Data
==========================

In [1]:
import pandas as pd  # programmers like shortening names for things, so they usually import pandas as "pd"
import altair as alt # again with the shortening of names
import requests # we use it for downloading the data so we don't have to save it on GitHub (too big!)
import zipfile # for de-compressing the files we download from the EPA

## Load the air quality data
The EPA published PM2.5 daily summary files annually [on their website](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily). This data is the "PM2.5 FRM/FEM Mass (88101)" dataset.
Pandas understands what a CSV file is, so here we can just load them into two `DataFrame`s. A data frame is simply one representation of a table of data. It is the most important form of storage for working with data in pandas.

In [9]:
# Download the data from the EPA website
data_file_urls = [
    'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2024.zip',
    'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2019.zip'
]
# copied this example from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
for url in data_file_urls:
    local_filename = "data/{}".format(url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
# and unzip the files
files_to_unzip = ["data/{}".format(url.split('/')[-1]) for url in data_file_urls]
for f in files_to_unzip:
    with zipfile.ZipFile(f,"r") as zip_ref:
        zip_ref.extractall("data")

In [10]:
air_2019_df = pd.read_csv("data/daily_88101_2019.csv", low_memory=False)
air_2020_df = pd.read_csv("data/daily_88101_2024.csv", low_memory=False)
air_2020_df.head() # this helpfully prints out the first few rows with headers to preview the data

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,NaN,209,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31
1,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,NaN,209,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31
2,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,NaN,209,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31
3,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,NaN,209,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31
4,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,NaN,209,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2024-10-31


In [11]:
"{} rows for 2019, {} rows for 2020".format(air_2019_df.shape[0], air_2019_df.shape[0])

'654349 rows for 2019, 654349 rows for 2020'

## Aggregate and average MA data by city
Let's compare MA data by city in 2020 and 2019.

In [16]:
# Step 1 - filter the data down by state name

is_MA_data = air_2020_df['State Name'] == "California"
air_MA_2020_df = air_2020_df[is_MA_data]

is_MA_data = air_2019_df['State Name'] == "California"
air_MA_2019_df = air_2019_df[is_MA_data]

"{} reports for MA in 2019, {} reports for MA in 2020".format(air_MA_2019_df.shape[0], air_MA_2020_df.shape[0])

'71619 reports for MA in 2019, 37430 reports for MA in 2020'

In [17]:
# now trim down to just the columns we care about so it is easier to understand
interesting_columns = ['City Name', 'Latitude', 'Longitude', 'Arithmetic Mean']
air_MA_2020_df = pd.DataFrame(air_MA_2020_df, columns=interesting_columns)
air_MA_2019_df = pd.DataFrame(air_MA_2019_df, columns=interesting_columns)
air_MA_2019_df

,City Name,Latitude,Longitude,Arithmetic Mean
22051,Livermore,37.687526,-121.784217,5.782609
22052,Livermore,37.687526,-121.784217,11.954545
22053,Livermore,37.687526,-121.784217,20.125000
22054,Livermore,37.687526,-121.784217,28.875000
22055,Livermore,37.687526,-121.784217,11.208333
...,...,...,...,...
93665,Woodland,38.661210,-121.732690,18.100000
93666,Woodland,38.661210,-121.732690,12.500000
93667,Woodland,38.661210,-121.732690,23.800000
93668,Woodland,38.661210,-121.732690,1.000000


In [18]:
# now group all the records by city and average them
avg_by_city_2020_MA_df = air_MA_2020_df.groupby('City Name').mean()\
    .reset_index()\
    .rename(columns={'City Name': 'City', 'Arithmetic Mean': 'Mean'})
avg_by_city_2019_MA_df = air_MA_2019_df.groupby('City Name').mean()\
    .reset_index()\
    .rename(columns={'City Name': 'City', 'Arithmetic Mean': 'Mean'})
# now we need to add in a year column so we can tell the data apart!
avg_by_city_2020_MA_df['year'] = 2020
avg_by_city_2019_MA_df['year'] = 2019
# now we can just contacetane the two dataframes to get all our data in one place
ma_city_avg_df = pd.concat([avg_by_city_2019_MA_df, avg_by_city_2020_MA_df])
ma_city_avg_df.to_csv('data/MA-city-year-avg.csv')
ma_city_avg_df

,City,Latitude,Longitude,Mean,year
0,Anaheim,33.830620,-117.938450,9.186568,2019
1,Arden-Arcade,38.613779,-121.368014,8.234511,2019
2,Arroyo Grande,35.046730,-120.587770,6.073006,2019
3,Atascadero,35.494530,-120.666170,4.217471,2019
4,Auburn,38.935680,-121.099590,7.185939,2019
...,...,...,...,...,...
87,Vallejo,38.102507,-122.237976,4.610483,2020
88,Victorville,34.510961,-117.325540,7.486910,2020
89,Visalia,36.308150,-119.312900,9.010281,2020
90,Woodland,38.661210,-121.732690,6.297114,2020


In [19]:
alt.Chart(ma_city_avg_df, height=300).mark_bar().encode(
    alt.X('year:N'),
    alt.Y('Mean'),
    color='year:N',
    column=alt.Column(field='City', type='ordinal', spacing=10)
).properties( 
    title="MA City Average PM2.5 (by year)",
)

alt.Chart(...)